In [57]:
import cv2
import numpy as np
import pandas as pd
import torch

from matplotlib import pyplot as plt
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, Dataset, random_split

In [58]:
import logging

logger = logging.getLogger("torch")
logger.setLevel(logging.CRITICAL)

In [59]:
logger.info('GPU: ' + str(torch.cuda.is_available()))

In [66]:
BATCH_SIZE = 16
N_EPOCH = 128
DEVICE = torch.device('cuda:0')

In [206]:
class HacksDataset(Dataset):
    def __init__(self, train_path, target_path):
        self.X = pd.read_csv(train_path).values
        self.labels = pd.read_csv(target_path).values

    def __len__(self):
        self.__len__ = len(self.labels)
        return self.__len__

    def __getitem__(self, idx):
        return self.X[idx], self.labels[idx]

In [207]:
dataset = HacksDataset(train_path="data/classifier_train.csv", target_path="data/classifier_target.csv")

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

trainloader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [212]:
class ThatModel(nn.Module):
    def __init__(self):
        super(ThatModel, self).__init__()

        self.classifier = nn.Sequential(
            nn.Linear(21, 64),
            nn.ReLU(),

            nn.Linear(64, 128),
            nn.ReLU(),

            nn.Linear(128, 69)
        )

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.classifier(x)

        return torch.argmax(torch.sigmoid(x), axis=1)

In [229]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim=1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim=1)

    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)

    acc = torch.round(acc * 100)

    return acc

In [243]:
model = ThatModel().to(DEVICE)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.1, weight_decay=0.005)


for epoch in range(N_EPOCH):
    running_loss = 0
    model.train()
    for idx, data in enumerate(trainloader):
        train_data, labels = data
        train_data = train_data.to(DEVICE).to(torch.float32)
        labels = labels.to(DEVICE).to(torch.float32)

        optimizer.zero_grad()

        outputs = model(train_data).view(labels.shape).to(torch.float32)
        outputs.requires_grad = True
        labels.requires_grad = True

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # if idx % 50 == 0:
        #     print(f"epoch: {epoch + 1} batch №: {idx + 1} loss: {round(loss.item(), 3)}", end="\r")

    running_loss /= idx
    accuracy = multi_acc(outputs, labels).item()
    print(f"{epoch=} {running_loss=:.2f} {accuracy=}", end="\r")

KeyboardInterrupt: 

In [244]:
labels.grad

tensor([[ 0.8750],
        [ 2.5000],
        [-2.7500],
        [-5.5000],
        [ 2.6250],
        [ 0.8750],
        [-6.0000],
        [ 2.0000],
        [-4.8750],
        [ 0.3750],
        [-6.3750],
        [ 1.2500],
        [-0.5000],
        [ 0.7500],
        [-0.2500],
        [-3.8750]], device='cuda:0')

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0', requires_grad=True)